In [2]:
import os
from os import walk
import gc
import warnings
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re

import mlflow
import mlflow.sklearn

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import mixed_precision
from tensorflow.keras.layers.experimental import preprocessing

import optuna

from sklearn.metrics import (
    mean_squared_error,
    mean_absolute_error,
    r2_score,
    confusion_matrix,
    classification_report,
    accuracy_score
)
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

import lightgbm
from lightgbm import LGBMRegressor

In [3]:
#mixed_precision.set_global_policy('mixed_float16')
# physical_devices = tf.config.list_physical_devices('GPU') 
# tf.config.experimental.set_memory_growth(physical_devices[0], True)

## Loading and processing data

In [4]:
df = pd.read_csv('project1_output.csv')

In [5]:
# we'll get a subset of our dataset in order to make experiments faster
df = df[:1000]
df = df.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))

#### Converting cols to their appropriate types again because we lost it on the csv export

In [6]:
df["product_category_name"] = df["product_category_name"].astype('category')
df["order_status"] = df["order_status"].astype('category')
df["review_score"] = df["review_score"].astype('category')
df["payment_type"] = df["payment_type"].astype('category')
df["customer_zip_code_prefix"] = df["customer_zip_code_prefix"].astype('category')
df["customer_city"] = df["customer_city"].astype('category')
df["customer_state"] = df["customer_state"].astype('category')
df["seller_zip_code_prefix"] = df["seller_zip_code_prefix"].astype('category')
df["seller_city"] = df["seller_city"].astype('category')
df["seller_state"] = df["seller_state"].astype('category')

df["product_name_lenght"] = df["product_name_lenght"].astype('int64')
df["product_description_lenght"] = df["product_description_lenght"].astype('int64')
df["product_photos_qty"] = df["product_photos_qty"].astype('int64')
df["payment_installments"] = df["payment_installments"].astype('int64')
df["payment_sequential"] = df["payment_sequential"].astype('int64')

In [7]:
df.dtypes

order_id                           object
order_item_id                       int64
product_id                         object
seller_id                          object
shipping_limit_date                object
price                             float64
freight_value                     float64
product_category_name            category
product_name_lenght                 int64
product_description_lenght          int64
product_photos_qty                  int64
product_weight_g                  float64
product_length_cm                 float64
product_height_cm                 float64
product_width_cm                  float64
customer_id                        object
order_status                     category
order_purchase_timestamp           object
order_approved_at                  object
order_delivered_carrier_date       object
order_delivered_customer_date      object
order_estimated_delivery_date      object
review_id                          object
review_score                     c

#### Droping unnecessary columns
We only want to work with numerical values.

In [8]:
df = df.select_dtypes(exclude=['object'])

In [9]:
df.columns

Index(['order_item_id', 'price', 'freight_value', 'product_category_name',
       'product_name_lenght', 'product_description_lenght',
       'product_photos_qty', 'product_weight_g', 'product_length_cm',
       'product_height_cm', 'product_width_cm', 'order_status', 'review_score',
       'payment_sequential', 'payment_type', 'payment_installments',
       'payment_value', 'customer_zip_code_prefix', 'customer_city',
       'customer_state', 'seller_zip_code_prefix', 'seller_city',
       'seller_state', 'payment_value_norm', 'volume'],
      dtype='object')

We'll also drop the `payment_value` column because our model would simply infer our target value from it by subtracting it from the `price` column.

In [10]:
df = df.drop(columns=['payment_value'])

### Feature engineering

Here we'll one-hot encode all of our categorical columns, and then drop the original ones

In [11]:
df = pd.get_dummies(df)
df = df.select_dtypes(exclude=['category'])

Even though we generated over 22000 columns this way, we believe that our model will be powerful enough to filter out any unecessary data.

## Picking column for prediction

We chose the `freight_value` column so we can perform a regression in order to try to find it's value based on all of the columns we have available.

In [12]:
TARGET_VALUE = 'freight_value'

In [13]:
target_col = df[TARGET_VALUE]

In [14]:
target_col

0      13.29
1      19.93
2      17.87
3      12.79
4      18.14
       ...  
995    74.99
996    34.98
997    17.03
998    19.07
999    15.59
Name: freight_value, Length: 1000, dtype: float64

In [15]:
df = df.drop(columns=[TARGET_VALUE])

## Separating prediction and test data

We'll split our data in a 60/20/20 ratio.

In [16]:
def get_x_data():
    # input 
    train, val, test = np.split(df.sample(frac=1), [int(.6*len(df)), int(.8*len(df))])
    
    return train, val, test

In [17]:
def get_y_data():
    # output
    train_labels, val_labels, test_labels = (
        np.split(
            target_col, 
            [int(.6*len(target_col)), int(.8*len(target_col))])
    )
    
    return train_labels, val_labels, test_labels

In [18]:
# deleting our initial df so we can free up some RAM
# del df

# Picking 4 ML algorithms

We'll use the following 4 algorithms:

1. Linear regression
2. Multilayer perceptron (a shallow one)
3. random forests
4. lightgbm/xgboost

### Metrics function

In [19]:
# Evaluate metrics
def eval_metrics(actual, pred):
    rmse = np.sqrt(mean_squared_error(actual, pred))
    mae = mean_absolute_error(actual, pred)
    r2 = r2_score(actual, pred)
    return rmse, mae, r2

### Enabling MLFlow autologging

In [20]:
mlflow.sklearn.autolog()
mlflow.tensorflow.autolog()
mlflow.lightgbm.autolog()

2021/08/15 23:34:25 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of tensorflow. If you encounter errors during autologging, try upgrading / downgrading tensorflow to a supported version, or try upgrading MLflow.


## Linear regression
Let's start off with linear regression, which is the most simple algorithm in our selection, and will serve as a baseline for the following algorithms.

In [21]:
def linear_regression(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)

    # Start an MLflow run; the "with" keyword ensures we'll close the run even if this cell crashes
    with mlflow.start_run(run_name="Linear Regression"):
        reg = LinearRegression()
        reg.fit(train, train_labels)

        predictions = reg.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("Linear regression model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        #mlflow.sklearn.log_model(reg, "model")
        #modelpath = "./mlflow/freight_value/model-linear-reg"
        #mlflow.sklearn.save_model(reg, modelpath)
        
        gc.collect()
        
        return rmse

In [22]:
study = optuna.create_study()
study.optimize(linear_regression, n_trials=1)

[I 2021-08-15 23:34:25,954] A new study created in memory with name: no-name-dab5cdf6-d3af-4a3d-bc5e-cd04a2250b40
[I 2021-08-15 23:34:26,940] Trial 0 finished with value: 90.99431207401365 and parameters: {}. Best is trial 0 with value: 90.99431207401365.


Linear regression model
  RMSE: 90.99431207401365
  MAE: 65.309461906497
  R2: -38.3482098714733


## Multilayer Perceptron

In [23]:
def mlp(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    # hyper-parameters to test
    params = {
        "hidden_units": trial.suggest_int("hidden_units", 3, 15),
        "lr": trial.suggest_float("lr", 1e-5, 1e-3, log=True),
        "epochs": trial.suggest_int("epochs", 10, 50)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    # Start an MLflow run
    with mlflow.start_run(run_name="MLP"):
        normalizer = preprocessing.Normalization(axis=-1)
        normalizer.adapt(np.array(train))
        
        mlp_model = tf.keras.Sequential([
            normalizer,
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=params["hidden_units"]),
            layers.Dense(units=1),
        ])

        mlp_model.summary()
        
        mlp_model.compile(
            optimizer=tf.optimizers.Adam(learning_rate=params["lr"]),
            loss='mean_squared_error'
        )

        history = mlp_model.fit(
            train, train_labels,
            validation_data=(test, test_labels),
            epochs=params["epochs"]
        )
        
        predictions = mlp_model.predict(val)

        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        # Print out model metrics
        print("MLP model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_name":"MultiLayerPerceptron",
                "estimator_class":"Keras"
            }
        )
        #mlflow.tensorflow.log_model(mlp_model, "model")
        #modelpath = "./mlflow/freight_value/model-mlp"
        #mlflow.tensorflow.save_model(mlp_model, modelpath)
        tf.keras.backend.clear_session()

        gc.collect()
        
        return rmse

### Using optuna to optimize MLP's hyperparameters

In [24]:
study = optuna.create_study()
study.optimize(mlp, n_trials=10)

[I 2021-08-15 23:34:26,967] A new study created in memory with name: no-name-3da30d36-2ebb-49f4-a393-e89192289a9f


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 1855)              3711      
_________________________________________________________________
dense (Dense)                (None, 13)                24128     
_________________________________________________________________
dense_1 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_2 (Dense)              (None, 13)                182       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 14        
Total params: 28,217
Trainable params: 24,506
Non-trainable params: 3,711
_________________________________________________________________
Epoch 1/48
19/19 [==============================] - 1s 15ms/step - loss: 540.9018 - val_loss: 514671968256.0000
Ep

[I 2021-08-15 23:34:32,260] Trial 0 finished with value: 762881.3161777382 and parameters: {'hidden_units': 13, 'lr': 1.2645740678991583e-05, 'epochs': 48}. Best is trial 0 with value: 762881.3161777382.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 1855)              3711      
_________________________________________________________________
dense (Dense)                (None, 8)                 14848     
_________________________________________________________________
dense_1 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_2 (Dense)              (None, 8)                 72        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 9         
Total params: 18,712
Trainable params: 15,001
Non-trainable params: 3,711
_________________________________________________________________
Epoch 1/47
19/19 [==============================] - 1s 13ms/step - loss: 536.6121 - val_loss: 73894846464.0000
Epo

[I 2021-08-15 23:34:36,799] Trial 1 finished with value: 243418.12581651183 and parameters: {'hidden_units': 8, 'lr': 1.3018700865964896e-05, 'epochs': 47}. Best is trial 1 with value: 243418.12581651183.


...

[I 2021-08-15 23:34:58,486] Trial 7 finished with value: 414396.67002250714 and parameters: {'hidden_units': 7, 'lr': 0.0001882715981204829, 'epochs': 10}. Best is trial 1 with value: 243418.12581651183.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 1855)              3711      
_________________________________________________________________
dense (Dense)                (None, 10)                18560     
_________________________________________________________________
dense_1 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_2 (Dense)              (None, 10)                110       
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 11        
Total params: 22,502
Trainable params: 18,791
Non-trainable params: 3,711
_________________________________________________________________
Epoch 1/45
19/19 [==============================] - 1s 14ms/step - loss: 535.5904 - val_loss: 145451270144.0000
Ep

[I 2021-08-15 23:35:02,935] Trial 8 finished with value: 587205.4544958103 and parameters: {'hidden_units': 10, 'lr': 4.871522981572731e-05, 'epochs': 45}. Best is trial 1 with value: 243418.12581651183.


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
normalization (Normalization (None, 1855)              3711      
_________________________________________________________________
dense (Dense)                (None, 5)                 9280      
_________________________________________________________________
dense_1 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_2 (Dense)              (None, 5)                 30        
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 6         
Total params: 13,057
Trainable params: 9,346
Non-trainable params: 3,711
_________________________________________________________________
Epoch 1/22
19/19 [==============================] - 1s 13ms/step - loss: 539.4087 - val_loss: 76868624384.0000
Epoc

[I 2021-08-15 23:35:06,035] Trial 9 finished with value: 1198343.9028469082 and parameters: {'hidden_units': 5, 'lr': 0.0004429587786509501, 'epochs': 22}. Best is trial 1 with value: 243418.12581651183.


## Random Forest

In [25]:
def random_forest(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "max_depth": trial.suggest_int("max_depth", 3, 10),
        "min_samples_split": trial.suggest_int("min_samples_split", 2, 5),
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Random Forest"):
        rf = RandomForestRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
            min_samples_split=params["min_samples_split"],
            random_state=0
        )
        rf.fit(train, train_labels)
        
        predictions = rf.predict(val)
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)
        
        print("Random Forest model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)
        
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        
        gc.collect()
        
        return rmse

### Using optuna to optimize Random Forest's hyperparameters

In [26]:
study = optuna.create_study()
study.optimize(random_forest, n_trials=10)

[I 2021-08-15 23:35:06,054] A new study created in memory with name: no-name-a12e8918-f381-4cd1-af42-eded737eec6d
[I 2021-08-15 23:35:07,671] Trial 0 finished with value: 15.028769774006358 and parameters: {'n_estimators': 134, 'max_depth': 4, 'min_samples_split': 4}. Best is trial 0 with value: 15.028769774006358.


Random Forest model
  RMSE: 15.028769774006358
  MAE: 9.24944312378477
  R2: -0.07335491700964591


[I 2021-08-15 23:35:09,357] Trial 1 finished with value: 15.616548157282168 and parameters: {'n_estimators': 83, 'max_depth': 9, 'min_samples_split': 5}. Best is trial 0 with value: 15.028769774006358.


Random Forest model
  RMSE: 15.616548157282168
  MAE: 9.487153705419797
  R2: -0.15895500840076604


[I 2021-08-15 23:35:10,631] Trial 2 finished with value: 15.302392666498529 and parameters: {'n_estimators': 61, 'max_depth': 7, 'min_samples_split': 2}. Best is trial 0 with value: 15.028769774006358.


...

Random Forest model
  RMSE: 15.583816889660739
  MAE: 9.492343794300764
  R2: -0.154101911225468
Random Forest model
  RMSE: 14.869604418574337
  MAE: 9.190870189117215
  R2: -0.050740124708194445


[I 2021-08-15 23:35:18,658] Trial 7 finished with value: 14.869604418574337 and parameters: {'n_estimators': 89, 'max_depth': 3, 'min_samples_split': 2}. Best is trial 7 with value: 14.869604418574337.
[I 2021-08-15 23:35:20,508] Trial 8 finished with value: 15.641466298625234 and parameters: {'n_estimators': 91, 'max_depth': 9, 'min_samples_split': 4}. Best is trial 7 with value: 14.869604418574337.


Random Forest model
  RMSE: 15.641466298625234
  MAE: 9.476235779929917
  R2: -0.16265647231419078


[I 2021-08-15 23:35:21,669] Trial 9 finished with value: 14.989840814110146 and parameters: {'n_estimators': 74, 'max_depth': 4, 'min_samples_split': 2}. Best is trial 7 with value: 14.869604418574337.


Random Forest model
  RMSE: 14.989840814110146
  MAE: 9.24752355687862
  R2: -0.06780150526396178


## Gradient Boosting with LightGBM

In [28]:
def gradient_boosting(trial):
    train, test, val = get_x_data()
    train_labels, val_labels, test_labels = get_y_data()
    
    params = {
        "n_estimators": trial.suggest_int("n_estimators", 50, 150),
        "num_leaves": trial.suggest_int("num_leaves", 25, 35),
        "max_depth": trial.suggest_int("max_depth", 3, 10)
    }
    
    warnings.filterwarnings("ignore")
    np.random.seed(40)
    
    with mlflow.start_run(run_name="Gradient Boosting"):
#         model = LGBMRegressor(
#             max_depth=params["max_depth"],
#             n_estimators=params["n_estimators"],
#             num_leaves=params["num_leaves"],
#         )
        model = XGBRegressor(
            max_depth=params["max_depth"],
            n_estimators=params["n_estimators"],
        )
        model.fit(train, train_labels)
        
        predictions = model.predict(test)
        print('Prediction: %.3f' % predictions[0])
        
        (rmse, mae, r2) = eval_metrics(val_labels, predictions)

        print("LGBM model")
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log mlflow attributes for mlflow UI
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("r2", r2)
        mlflow.log_metric("mae", mae)
        mlflow.log_params(trial.params)
        mlflow.set_tags(
            {
                "estimator_class":"LightGBM",
                "estimator_name":"Gradient Boosting"
            }
        )
        mlflow.sklearn.log_model(model, "model")
        
        gc.collect()
        
        return rmse

### Using optuna to optimize Gradient Boosting's hyperparameters

In [29]:
study = optuna.create_study()
study.optimize(gradient_boosting, n_trials=10)

[I 2021-08-15 23:36:06,187] A new study created in memory with name: no-name-b6c9b334-f886-4dd1-b657-7d00750392df
[I 2021-08-15 23:36:09,574] Trial 0 finished with value: 16.125656800791422 and parameters: {'n_estimators': 110, 'num_leaves': 32, 'max_depth': 5}. Best is trial 0 with value: 16.125656800791422.


Prediction: 26.947
LGBM model
  RMSE: 16.125656800791422
  MAE: 10.054394902801514
  R2: -0.23575197196477937


[I 2021-08-15 23:36:12,045] Trial 1 finished with value: 15.68434129076245 and parameters: {'n_estimators': 83, 'num_leaves': 29, 'max_depth': 4}. Best is trial 1 with value: 15.68434129076245.


Prediction: 20.498
LGBM model
  RMSE: 15.68434129076245
  MAE: 9.609959377098084
  R2: -0.16903914856935187


[I 2021-08-15 23:36:13,564] Trial 2 finished with value: 15.294295479565868 and parameters: {'n_estimators': 57, 'num_leaves': 35, 'max_depth': 3}. Best is trial 2 with value: 15.294295479565868.


...

[I 2021-08-15 23:36:34,660] Trial 7 finished with value: 16.096463090082608 and parameters: {'n_estimators': 100, 'num_leaves': 35, 'max_depth': 5}. Best is trial 2 with value: 15.294295479565868.


Prediction: 26.178
LGBM model
  RMSE: 16.096463090082608
  MAE: 10.0064317401886
  R2: -0.23128163874871466


[I 2021-08-15 23:36:43,535] Trial 8 finished with value: 16.878638207733626 and parameters: {'n_estimators': 140, 'num_leaves': 34, 'max_depth': 10}. Best is trial 2 with value: 15.294295479565868.


Prediction: 30.312
LGBM model
  RMSE: 16.878638207733626
  MAE: 10.67378908352852
  R2: -0.3538523261100506
Prediction: 30.309
LGBM model
  RMSE: 16.875874308867214
  MAE: 10.67277538971901
  R2: -0.35340897226929746


[I 2021-08-15 23:36:52,312] Trial 9 finished with value: 16.875874308867214 and parameters: {'n_estimators': 139, 'num_leaves': 35, 'max_depth': 10}. Best is trial 2 with value: 15.294295479565868.


## Selecting best model